# Pelle's Rotor-Supported Arithmetic (crypto)

This is a standard RSA challenge, with the added bonus that we can query something that manipulates $d$. Specifically, you can provide a base $c$ and a rotation amount $i$, and it will rotate the decimal representation of $d$ left by $i$ steps, then the oracle will tell you `pow(c, rotor(d, i), n)`. The challenge is thus to leak $d$.

The first step is to leak $n$, which we can do by letting `(c,i)=(-1,0)`.

The next idea is to notice that you can link two oracle values in this way:
- Suppose you have a smaller example with $d=314159$, and make two queries `(2,0)` and `(2,1)`.
- You get $r = 2^{314159}$ and $s = 2^{141593}$.
- Then you have two ways of forming $2^{3141593}$, namely $r^{10} \cdot 2^3 = s \cdot 2^{3 \times 10^6}$.
- The "3" here is an unknown, but by knowing `r` and `s` you can just brute-force (locally) all 10 possible digits.
- Additionally, in the first step you don't know the length of $d$, but from the challenge it's likely to be between 306 and 309 inclusive.

So the plan is that the first step is slightly heavier because you have to guess the length as well, and subsequent steps are shorter because you already know the length. Now, because 10 is such a small number to brute force, we instead choose to skip 2 digits at a time and brute force over 100 possibilities. This is just an aesthetic decision and doesn't affect the solution.

In [1]:
from pwn import *
from tqdm import trange
from Crypto.Util.number import long_to_bytes
from math import isqrt

In [2]:
sh = remote('pelle-01.hfsc.tf', 4591)
sh.recvuntil(b'encrypted flag ')
ef = int(sh.recvline())
print(f'{ef=}')

def get(c,i):
    sh.sendline(f'1\n{c}\n{i}'.encode())
    sh.recvuntil(b'rot:\n')
    return int(sh.recvline())
n=get(-1,0)+1
print(f'{n=}')
ps = [get(2,2*i) for i in trange(160)]

[x] Opening connection to pelle-01.hfsc.tf on port 4591
[x] Opening connection to pelle-01.hfsc.tf on port 4591: Trying 159.65.24.100
[+] Opening connection to pelle-01.hfsc.tf on port 4591: Done
ef=38472369479330817284470725572557699354671892874140360225093249123807422689575697431459769243490515430225070907323779024621916267683075124489839093003363458348819521180068938611239726138319523799056746792085184682064924292637267952233681355598069903759048181691592104467133991048147706829863953183279851392375
n=43047796890477362990074961769201922093931501549521114743916627406636416622979445051218421149675256799232393301700370094540087679119082880899747691935251456178408503271262210863920917237676311115019888447360967178340255706657371448083420218420057544839628399548904356647638728064251771091862957676254700954117


100%|██████████| 160/160 [00:26<00:00,  6.12it/s]


In [3]:
def solve(dls, diff, p0, p1):
    if isinstance(dls, int):
        dls = [dls]
    for dl in dls:
        for hi in range(0, 10**diff):
            t0 = pow(p0,10**diff,n)*pow(2,hi,n)
            t1 = p1*pow(2,hi*10**dl,n)
            if (t0 - t1) % n == 0:
                return hi, dl
            
soln0, dl = solve([306,307,308,309], 2, ps[0], ps[1])
print(f'{dl=}') # length of d
solns = [solve(dl, 2, ps[i], ps[i+1])[0] for i in trange(1,154)]
d=int(''.join(f'{i:02}' for i in [soln0]+solns)[:dl])
print(f'{d=}')

dl=308


100%|██████████| 153/153 [00:21<00:00,  7.16it/s]

d=27674287480195801722658014392785989313845949971533997686660899124104025285859437857972359428690936714607919108095274315625722478853856599143509311634893475488768721169431521077251366133165140420429800291880801747490176817398575093644983093206936398362605552523502972162034264567777255223024221794336695299713


Hooray, we've successfully leaked $d$. All that's left is to get $p$ and $q$ back out of this, so that we can get our flag.

In [4]:
e=65537
k=int(d*e/n+0.5)
b=(n+1-(d*e-1)//k)//2

p=b-isqrt(b**2-n)
q=b+isqrt(b**2-n)
assert p*q==n

long_to_bytes(pow(ef, pow(3331646268016923629, -1, (p-1)*(q-1)), n))

b'midnight{twist_and_turn}'